## ANJUMSO

Welcome to **ANJUMSO** - Annotated Jupyter Modelling Software.

In order to run the code, please, install Anaconda 3.8: https://www.anaconda.com/products/individual

After the installation, download the given repository and unpack it on your PC. In the folder you will find the
following files:

1. ANJUMSO.ipynb - JuPyTer notebook, that includes model and auxiliary functions
2. Model Parameters.xlsx - Parameters of the model and model initial conditions that are stored as the Excel file.
You can edit it and this will result in the parameter change in the model. Each module of the model has its own
parameters and they are stored on the corresponding sheets of the Excel file
3. Exp_data.xlsx - ANJUMSO can read and display experimental data for the comparison with the modeling results

Excel files are to be stored in the same folder as the model.

The example model is the model of platelet collagen receptor GPVI and is based on the models from:
Martyanov *et. al.* The Biophys. J. 2020; Garzon Dasgupta *et. al.* Life 2020; Sveshnikova *et. al.* JTH 2016.
Model validation data were taken from: Dunster *et. al.* PLOS Comp. Biol. 2015; Poulter *et. al.* JTH 2017.

To run the model - execute all of the code cells. You can change the parameters of the model dynamically
by passing the model the name of the module, the name of the parameter and the range of the parameter variation.
You can also change the variables that are displayed in the plots by changing of the corresponding variable number.
The instructions for these are given in the corresponding code parts.

Parameter and variable description is given in the excel files.

Enjoy!

In [ ]:
# Python libraries, used in the model

import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets

from ipywidgets import interact
from scipy.integrate import solve_ivp

In [ ]:
# Model definition

def GPVI_model(time, z, clustering_pars, kinases_pars, pars1, LAT_pars, calcium_pars):

    # Model variable unpacking. Variables have to be in the same order, as they would be returned by the model.
    # This is critical

    GPVI, GPVI_Act, GPVI_Clust, Ip, Syk, Syk_A, LAT, LAT_p, PIP2, PIP3, PLCg2, PLCg2_A, IP3, \
        Ca_cyt, Ca_dts, Ca_buf, buf,\
        IP3R_n, IP3R_o, IP3R_a, IP3R_i1, IP3R_i2, IP3R_s,\
        Serca2b_e1, Serca2b_e1_ca, Serca2b_e1_2ca, Serca2b_e2, Serca2b_e1_2ca_p, Serca2b_e2_2ca_p, Serca2b_e2_p = z

    GPVI_0 = pars1[0]
    CRP = kinases_pars['Pars_Horm']['CRP']

    # GPVI clustering module parameters

    kf_act = clustering_pars['Pars_Horm']['Kf_Ligation']
    kr_act = clustering_pars['Pars_Horm']['Kr_Ligation']
    k1 = clustering_pars['Pars_Horm']['k1']
    k2 = clustering_pars['Pars_Horm']['k2']
    k3 = clustering_pars['Pars_Horm']['k3']
    k_1 = clustering_pars['Pars_Horm']['k_1']
    k_2 = clustering_pars['Pars_Horm']['k_2']

    # GPVI clustering module equations

    dGPVIdt = - kf_act * CRP * GPVI + kr_act * GPVI_Act
    dGPVI_Actdt = kf_act * CRP * GPVI - kr_act * GPVI_Act - k1 * GPVI_Clust * GPVI_Act + k_1 * GPVI_Clust - \
                  2 * k2 * GPVI_Act * GPVI_Act
    dGPVI_Clustdt = - (k_2 * GPVI_Clust + k3) * GPVI_Clust + k2 * GPVI_Act * GPVI_Act

    # Tyrosine kinase activation module parameters

    kcat_SFK_A = kinases_pars['Pars_Horm']['kcat_SFK']
    kr_P = kinases_pars['Pars_Horm']['kr_P']
    kcat_CD148 = kinases_pars['Pars_Horm']['kcat_CD148']

    pp_act = kinases_pars['Pars_Horm']['pp_act']
    A = kinases_pars['Pars_Horm']['Monomer_Size'] * 1e-18
    kf_Syk = kinases_pars['Pars_Horm']['kf_Syk']
    kr_Syk = kinases_pars['Pars_Horm']['kr_Syk']
    CD148 = kinases_pars['Pars_Horm']['CD148']

    S = clustering_pars['Pars_Horm']['S']
    V = clustering_pars['Pars_Horm']['V']
    NA = clustering_pars['Pars_Horm']['Avog']

    # Tyrosine kinase activation module equations

    if GPVI_Clust * S * NA < 1:
        Size = 1
        SFK_A = 0
    else:
        Size = (GPVI_0 - (GPVI_Act + GPVI)) / GPVI_Clust
        if math.isnan(Size):
            Size = 1
        if math.isinf(Size):
            Size = 1
        if abs(Size) > 1000:
            Size = 1
        SFK_A = ((kcat_CD148 / kcat_SFK_A) * Size * (CD148 * pp_act * GPVI_Clust) / (GPVI_Act + GPVI)) ** (1 / 2)

    dIp = kcat_SFK_A * (SFK_A / (Size * A * NA)) * (1 - Ip) / (A * NA) - kr_P * Ip

    dSyk_A = kf_Syk * Ip / (A * NA) * Syk + SFK_A / (Size * A * NA) * kcat_SFK_A * Syk - kr_Syk * Syk_A
    dSyk = - kf_Syk * Ip / (A * NA) * Syk - SFK_A / (Size * A * NA) * kcat_SFK_A * Syk + kr_Syk * Syk_A

    # LAT module parameters

    kcat_Syk = kinases_pars['Pars_Horm']['kcat_Syk']
    KM_Syk = kinases_pars['Pars_Horm']['KM_Syk']
    kr_LAT = kinases_pars['Pars_Horm']['Kr_LAT']
    kD_Syk = kinases_pars['Pars_Horm']['KD_Syk']

    kcat_PI3K = LAT_pars['Pars_Horm']['kcat_PI3K']
    KM_PI3K = LAT_pars['Pars_Horm']['KM_PI3K']
    kr_PIP3 = LAT_pars['Pars_Horm']['kr_PIP3']
    kD_PI3K = LAT_pars['Pars_Horm']['kD_PI3K']
    PI3K = LAT_pars['Pars_Horm']['PI3K'] / (NA * V)

    kcat_Btk = LAT_pars['Pars_Horm']['kcat_Btk']
    KM_Btk = LAT_pars['Pars_Horm']['KM_Btk']
    kD_Btk = LAT_pars['Pars_Horm']['kD_Btk']
    kD_PLCg2 = LAT_pars['Pars_Horm']['kD_PLCg2']
    kr_PLCg2 = LAT_pars['Pars_Horm']['kr_PLCg2']
    Btk = LAT_pars['Pars_Horm']['Btk'] / (NA * V)

    # LAT module equations

    dLATp_dt = kcat_Syk * (Syk_A / (NA * V)) * LAT / (KM_Syk / kD_Syk) - kr_LAT * LAT_p
    dLAT_dt = (- kcat_Syk * (Syk_A / (NA * V)) * LAT / (KM_Syk / kD_Syk) + kr_LAT * LAT_p)

    dPLCg2_A = kcat_Btk * PIP3 * Btk * (S / V) ** 2 * \
               (PLCg2 * LAT_p / kD_PLCg2) / (KM_Btk / kD_Btk + (PLCg2 * LAT_p / kD_PLCg2)) - kr_PLCg2 * PLCg2_A
    dPLCg2 = - kcat_Btk * PIP3 * Btk * (S / V) ** 2 * \
             (PLCg2 * LAT_p / kD_PLCg2) / (KM_Btk / kD_Btk + (PLCg2 * LAT_p / kD_PLCg2)) - kr_PLCg2 * PLCg2_A

    # Calcium Module parameters

    V_IM = calcium_pars['Value']['V_IM']
    V_DTS = calcium_pars['Value']['V_DTS']
    kcat_PLCg2 = calcium_pars['Value']['kcat_PLCg2']
    KM_PLCg2 = calcium_pars['Value']['KM_PLCg2']
    kr_IP3 = calcium_pars['Value']['kr_IP3']
    kf_Buf = calcium_pars['Value']['kf_Buf']
    kr_Buf = calcium_pars['Value']['kr_Buf']

    O_1 = V_IM * 0.12 * 11.94 / (0.12 + Ca_cyt)
    O_2 = V_IM * 90.56
    O_3 = V_IM * 1.7768 / (0.12 + Ca_cyt)
    O_4 = V_IM * 0.84
    O_5 = V_IM * (37.4 * 0.025 + 1.7 * Ca_cyt) / (0.025 + Ca_cyt * 0.145 / 0.12)
    O_6 = V_IM * (1.4 + Ca_cyt * 2.5)
    O_7 = V_IM * 1.7768 / (0.12 + Ca_cyt * 0.145 / 0.12)
    IP3R_num = V_IM * (IP3R_a + IP3R_i1 + IP3R_i2 + IP3R_n + IP3R_o + IP3R_s)
    O_P0 = (V_IM * (0.9 * IP3R_a + 0.1 * IP3R_o) / IP3R_num)**5.5

    I_1 = V_IM * 160
    I_2 = V_IM * 940
    I_3 = V_IM * 1.6
    I_4 = V_IM * 900
    I_5 = V_IM * 250
    I_6 = V_IM * 8e-5
    I_7 = V_IM

    # Calcium Module equations

    dPIP3 = kcat_PI3K * LAT_p * (PI3K * (S / V)) * PIP2 / (KM_PI3K / kD_PI3K + PIP2) - kr_PIP3 * PIP3
    dPIP2 = - kcat_PI3K * LAT_p * (PI3K * (S / V)) * PIP2 / (KM_PI3K / kD_PI3K + PIP2) \
            + kr_PIP3 * PIP3 - \
            (kcat_PLCg2 * PLCg2_A * PIP2 * (S / V) / (KM_PLCg2 / kD_PLCg2 + PIP2) * Ca_cyt / (Ca_cyt + 0.2)
                 - kr_IP3 * IP3) / S

    dIP3 = (kcat_PLCg2 * PLCg2_A * PIP2 * (S / V) / (KM_PLCg2 / kD_PLCg2 + PIP2) * Ca_cyt / (Ca_cyt + 0.2)
                - kr_IP3 * IP3 + O_6 * IP3R_o - O_5 * IP3R_n * IP3) / V

    dCa_cyt = (0.1351 * I_7 * math.log(Ca_dts / Ca_cyt)
               + 800 * IP3R_num * O_P0 * math.log(Ca_dts / Ca_cyt)
               - O_2 * IP3R_o * Ca_cyt + O_1 * IP3R_a - O_7 * IP3R_n * Ca_cyt + O_4 * IP3R_i1
               - O_3 * IP3R_a * Ca_cyt + O_4 * IP3R_i2
               + I_1 * Serca2b_e1_ca - I_2 * Serca2b_e1 * Ca_cyt
               + I_3 * Serca2b_e1_2ca - I_4 * Serca2b_e1_ca * Ca_cyt
               - V * 4 * kf_Buf * Ca_cyt**4 * buf + V * 4 * kr_Buf * Ca_buf) / V

    dCa_dts = (- 0.1351 * I_7 * math.log(Ca_dts / Ca_cyt)
               - 800 * IP3R_num * O_P0 * math.log(Ca_dts / Ca_cyt)
               + 2 * I_5 * Serca2b_e2_2ca_p - 2 * I_6 * Serca2b_e2_p * Ca_dts**2) / V_DTS

    dbuf = (- V * 4 * kf_Buf * Ca_cyt**4 * buf + 4 * V * kr_Buf * Ca_buf) / V

    dCa_buf = (V * 4 * kf_Buf * Ca_cyt**4 * buf - V * 4 * kr_Buf * Ca_buf) / V

    dIP3R_n = (- O_5 * IP3R_n * IP3 + O_6 * IP3R_o + O_4 * IP3R_i1 - O_7 * IP3R_n * Ca_cyt) / V_IM

    dIP3R_o = (O_1 * IP3R_a - O_2 * IP3R_o * Ca_cyt + O_5 * IP3R_n * IP3 - O_6 * IP3R_o
                + V_IM * 29.8 * IP3R_s - V_IM * 0.11 * IP3R_o) / V_IM

    dIP3R_a = (- O_1 * IP3R_a + O_2 * IP3R_o * Ca_cyt + O_4 * IP3R_i2 - O_3 * IP3R_a * Ca_cyt) / V_IM

    dIP3R_i1 = (- O_4 * IP3R_i1 + O_7 * IP3R_n * Ca_cyt) / V_IM

    dIP3R_i2 = (- O_4 * IP3R_i2 + O_3 * IP3R_a * Ca_cyt) / V_IM

    dIP3R_s = (- V_IM * 29.8 * IP3R_s + V_IM * 0.11 * IP3R_o) / V_IM

    dSerca2b_e1 = (I_1 * Serca2b_e1_ca - I_2 * Serca2b_e1 * Ca_cyt
                   - V_IM * 200 * Serca2b_e1 + V_IM * 280 * Serca2b_e2) / V_IM

    dSerca2b_e1_2ca = (- I_3 * Serca2b_e1_2ca + I_4 * Serca2b_e1_ca * Ca_cyt
                       + V_IM * 60 * Serca2b_e1_2ca_p - V_IM * 600 * Serca2b_e1_2ca) / V_IM

    dSerca2b_e1_2ca_p = (- V_IM * 60 * Serca2b_e1_2ca_p + V_IM * 600 * Serca2b_e1_2ca
                         + V_IM * 25 * Serca2b_e2_2ca_p - V_IM * 65 * Serca2b_e1_2ca_p) / V_IM

    dSerca2b_e1_ca = (- I_1 * Serca2b_e1_ca + I_2 * Serca2b_e1 * Ca_cyt
                      + I_3 * Serca2b_e1_2ca - I_4 * Serca2b_e1_ca * Ca_cyt) / V_IM

    dSerca2b_e2 = (V_IM * 105 * Serca2b_e2_p - V_IM * 1.6 * Serca2b_e2
                   + V_IM * 200 * Serca2b_e1 - V_IM * 280 * Serca2b_e2) / V_IM

    dSerca2b_e2_2ca_p = (- V_IM * 25 * Serca2b_e2_2ca_p + V_IM * 65 * Serca2b_e1_2ca_p
                          - I_5 * Serca2b_e2_2ca_p + I_6 * Serca2b_e2_p * Ca_dts**2) / V_IM

    dSerca2b_e2_p = (I_5 * Serca2b_e2_2ca_p - I_6 * Serca2b_e2_p * Ca_dts**2
                     - V_IM * 105 * Serca2b_e2_p + V_IM * 1.6 * Serca2b_e2) / V_IM

    return [dGPVIdt, dGPVI_Actdt, dGPVI_Clustdt, dIp, dSyk,  # 0 - 4
            dSyk_A, dLAT_dt, dLATp_dt, dPIP2, dPIP3,  # 5 - 9
            dPLCg2, dPLCg2_A, dIP3,  # 10 - 12
            dCa_cyt, dCa_dts, dCa_buf, dbuf,  # 13 - 16
            dIP3R_n, dIP3R_o, dIP3R_a, dIP3R_i1, dIP3R_i2, dIP3R_s,
            dSerca2b_e1, dSerca2b_e1_ca, dSerca2b_e1_2ca, dSerca2b_e2, dSerca2b_e1_2ca_p,
            dSerca2b_e2_2ca_p, dSerca2b_e2_p]

In [1]:
# Model calculation and plotting

def plot_model(clust_pars, kin_pars, LAT_pars, calcium_pars,
               InitCond, time_calc, ssteps, varying_list):

    # clust_pars - clustering module parameters
    # kin_pars - kinase module parameters
    # LAT_pars - LAT module parameters
    # calcium_pars - Calcium module parameters

    NA = 6.02e17
    Volume = clust_pars['Pars_Horm']['V']
    Area = clust_pars['Pars_Horm']['S']

    GPVI_0 = InitCond['Value']['GPVI']
    GPVI_Act_0 = InitCond['Value']['GPVI_Act']
    GPVIClust_0 = InitCond['Value']['Clusters']

    Syk_A_0 = InitCond['Value']['Syk_A']
    Syk_0 = InitCond['Value']['Syk']
    Ip = InitCond['Value']['Ip']
    LAT = InitCond['Value']['LAT'] / (NA * Area)
    LAT_p = InitCond['Value']['LAT_p'] / (NA * Area)

    PIP2 = InitCond['Value']['PIP2'] / (NA * Area)
    PIP3 = InitCond['Value']['PIP3'] / (NA * Area)

    PLCg2 = InitCond['Value']['PLCg2'] / (NA * Volume)
    PLCg2_A = InitCond['Value']['PLCg2_A'] / (NA * Volume)

    IP3 = InitCond['Value']['IP3']
    Ca_cyt = InitCond['Value']['Ca_cyt']
    Ca_dts = InitCond['Value']['Ca_dts']
    Ca_buf = InitCond['Value']['Ca_buf']
    buf = InitCond['Value']['buf']
    IP3R_n = InitCond['Value']['IP3R_n']
    IP3R_o = InitCond['Value']['IP3R_o']
    IP3R_a = InitCond['Value']['IP3R_a']
    IP3R_i1 = InitCond['Value']['IP3R_i1']
    IP3R_i2 = InitCond['Value']['IP3R_i2']
    IP3R_s = InitCond['Value']['IP3R_s']
    Serca2b_e1 = InitCond['Value']['Serca2b_e1']
    Serca2b_e1_ca = InitCond['Value']['Serca2b_e1_ca']
    Serca2b_e1_2ca = InitCond['Value']['Serca2b_e1_2ca']
    Serca2b_e2 = InitCond['Value']['Serca2b_e2']
    Serca2b_e1_2ca_p = InitCond['Value']['Serca2b_e1_2ca_p']
    Serca2b_e2_2ca_p = InitCond['Value']['Serca2b_e2_2ca_p']
    Serca2b_e2_p = InitCond['Value']['Serca2b_e2_p']

    Initial = [GPVI_0, GPVI_Act_0, GPVIClust_0, Ip, Syk_0, Syk_A_0,
               LAT, LAT_p, PIP2, PIP3, PLCg2, PLCg2_A, IP3,
               Ca_cyt, Ca_dts, Ca_buf, buf,
               IP3R_n, IP3R_o, IP3R_a, IP3R_i1, IP3R_i2, IP3R_s,
               Serca2b_e1, Serca2b_e1_ca, Serca2b_e1_2ca, Serca2b_e2,
               Serca2b_e1_2ca_p, Serca2b_e2_2ca_p, Serca2b_e2_p]

    def slidercalc(**kwargs):

        keylist = list(kwargs.keys())

        for j in range(len(keylist)):
            if keylist[j].split('.')[0] == 'Clustering':
                clust_pars['Pars_Horm'][keylist[j].split('.')[1]] = \
                    kwargs[keylist[j]]
            if keylist[j].split('.')[0] == 'KinaseActivation':
                kin_pars['Pars_Horm'][keylist[j].split('.')[1]] = \
                    kwargs[keylist[j]]
            if keylist[j].split('.')[0] == 'LAT_Pars':
                LAT_pars['Pars_Horm'][keylist[j].split('.')[1]] = \
                    kwargs[keylist[j]]
            if keylist[j].split('.')[0] == 'calcium_pars':
                calcium_pars['Value'][keylist[j].split('.')[1]] = \
                    kwargs[keylist[j]]

        model_calc = solve_ivp(GPVI_model, [0, time_calc], Initial,
                           args=(clust_pars, kin_pars, Initial, LAT_pars, calcium_pars, ), max_step=100000,
                           dense_output=True, method='LSODA', rtol=1e-6, atol=1e-12)
        time = np.linspace(0, time_calc, ssteps)
        solution = model_calc.sol(time)

        transposed = list(map(list, zip(*solution.T)))

        size = [0, ]
        for i in range(1, len(transposed[0])):
            size.append((GPVI_0 - transposed[0][i] - transposed[1][i]) /
                        (transposed[2][i]))

        res1 = []
        res2 = []

        # print(InitCond['Value']['V'] * 6.02e17)
        for i in range(len(transposed[2])):
            res1.append(transposed[8][i] )
            res2.append(transposed[13][i])

        plotres2, = plt.plot(time, res2)

        data = pd.read_excel('Exp_data.xlsx', sheet_name='SykY525')

        timepoints = data['Time']
        val = data['Syk']

        # plt.ylim(1000, 20000)

        # plt.scatter(timepoints, val)

        plt.show()

    def gen_slider(input_list):
        if input_list[0] == 'KinaseActivation':
            return widgets.FloatSlider(
                value=kin_pars['Pars_Horm'][input_list[1]],
                min=input_list[2],
                max=input_list[3],
                step=(input_list[3] - input_list[2]) / 1000,
                description=input_list[1],
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.5f')
        elif input_list[0] == 'Clustering':
            return widgets.FloatSlider(
                value=clust_pars['Pars_Horm'][input_list[1]],
                min=input_list[2],
                max=input_list[3],
                step=(input_list[3] - input_list[2]) / 1000,
                description=input_list[1],
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.5f')
        if input_list[0] == 'LAT_Pars':
            return widgets.FloatSlider(
                value=LAT_pars['Pars_Horm'][input_list[1]],
                min=input_list[2],
                max=input_list[3],
                step=(input_list[3] - input_list[2]) / 1000,
                description=input_list[1],
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.5f')
        if input_list[0] == 'calcium_pars':
            return widgets.FloatSlider(
                value=calcium_pars['Value'][input_list[1]],
                min=input_list[2],
                max=input_list[3],
                step=(input_list[3] - input_list[2]) / 1000,
                description=input_list[1],
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.5f')

    weight_sliders = [gen_slider(varying_list[i])
                      for i in range(len(varying_list))]

    kwargs = {varying_list[i][0] + '.' + varying_list[i][1]:
              slider for i, slider in enumerate(weight_sliders)}

    interact(slidercalc, **kwargs)


In [ ]:
clustering_pars = pd.read_excel('Model_Parameters.xlsx', sheet_name='Clustering').set_index('Parname')
kinase_pars = pd.read_excel('Model_Parameters.xlsx', sheet_name='KinaseActivation').set_index('Parname')
InitialValues = pd.read_excel('Model_Parameters.xlsx', sheet_name='InitialConcentrations').set_index('Name')
LAT_pars = pd.read_excel('Model_Parameters.xlsx', sheet_name='LAT_Pars').set_index('Parname')
calcium_pars = pd.read_excel('Model_Parameters.xlsx', sheet_name='CalciumModule').set_index('Parname')

plot_model(clustering_pars, kinase_pars, LAT_pars, calcium_pars,
           InitialValues,
           time_calc=300, ssteps=1000,
           varying_list = [['KinaseActivation', 'Kr_LAT', 0.1, 1],
                           ['calcium_pars', 'kr_IP3', 0.001, 0.01],
                           ['calcium_pars', 'kcat_PLCg2', 0, 10],
                           ['LAT_Pars', 'kr_PIP3', 0.1, 10],
                           ['LAT_Pars', 'kr_PLCg2', 0.01, 0.2],
                           ['KinaseActivation', 'Monomer_Size', 1, 10]])